<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/Book_2/ch04/Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4 word2vec speed improvement

**4.1 word2vec improvement**

*4.1.2 Embedding layer implementation*

In [ ]:
import numpy as np
W = np.arange(21).reshape(7, 3)
W

In [ ]:
W[2]

In [ ]:
W[5]

In [ ]:
idx = np.array([1, 0, 3, 0])
W[idx]

In [ ]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0

        np.add.at(dW, self.idx, dout)

        return None